# Setup

In [ ]:
%%capture
!git clone https://github.com/speechbrain/benchmarks
%cd /content/benchmarks
!git checkout eeg

%cd /content/benchmarks/benchmarks/MOABB
!pip install -r extra-requirements.txt # Install additional dependencies

In [ ]:
%%capture
# Clone SpeechBrain repository (development branch)
%cd /content/
!git clone https://github.com/speechbrain/speechbrain/
%cd /content/speechbrain/

# Install required dependencies
!pip install -r requirements.txt

# Install SpeechBrain in editable mode
!pip install -e .

%cd /content/

# YAML

In [ ]:
MyVanillaNeuralNetworkYAML = """seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True
data_iterator_name: !PLACEHOLDER
target_subject_idx: !PLACEHOLDER
target_session_idx: !PLACEHOLDER
events_to_load: null
original_sample_rate: 250
sample_rate: 125
fmin: 0.13
fmax: 46.0
n_classes: 4
tmin: 0.
tmax: 4.0
n_steps_channel_selection: 2
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
test_with: 'last'
test_key: "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 1000
avg_models: 10
number_of_epochs: 1000
lr: 0.005
max_lr: !ref <lr>
base_lr: 0.00000001
step_size_multiplier: 5
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>
batch_size_exponent: 4
batch_size: !ref 2 ** <batch_size_exponent>
valid_ratio: 0.2

# DATA AUGMENTATION
max_num_segments: 3
cutcat: !new:speechbrain.augment.time_domain.CutCat
    min_num_segments: 2
    max_num_segments: !ref <max_num_segments>
amp_delta: 0.01742
rand_amp: !new:speechbrain.augment.time_domain.RandAmp
    amp_low: !ref 1 - <amp_delta>
    amp_high: !ref 1 + <amp_delta>
shift_delta_: 1
shift_delta: !ref 1e-2 * <shift_delta_>
min_shift: !apply:math.floor
    - !ref 0 - <sample_rate> * <shift_delta>
max_shift: !apply:math.floor
    - !ref 0 + <sample_rate> * <shift_delta>
time_shift: !new:speechbrain.augment.freq_domain.RandomShift
    min_shift: !ref <min_shift>
    max_shift: !ref <max_shift>
    dim: 1
snr_white_low: 15.0
snr_white_delta: 19.1
snr_white_high: !ref <snr_white_low> + <snr_white_delta>
add_noise_white: !new:speechbrain.augment.time_domain.AddNoise
    snr_low: !ref <snr_white_low>
    snr_high: !ref <snr_white_high>
repeat_augment: 1
augment: !new:speechbrain.augment.augmenter.Augmenter
    parallel_augment: True
    concat_original: True
    parallel_augment_fixed_bs: True
    repeat_augment: !ref <repeat_augment>
    shuffle_augmentations: True
    min_augmentations: 4
    max_augmentations: 4
    augmentations: [
        !ref <cutcat>,
        !ref <rand_amp>,
        !ref <time_shift>,
        !ref <add_noise_white>]

# DATA NORMALIZATION
dims_to_normalize: 1
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

# MODEL
input_shape: [1, !ref <T>, !ref <C>, 1]
hidden_layers: [64,32]
activation_type: 'leaky_relu' # @orion_step1: --activation_type~"choices(['relu','leaky_relu','gelu'])"
dropout: 0.3563 # @orion_step1: --dropout~"uniform(0.0, 0.5)"

model: !new:models.MyVanillaNeuralNet.MyVanillaNeuralNet
    input_shape: !ref <input_shape>
    hidden_layers: !ref <hidden_layers>
    dropout: !ref <dropout>
    n_neurons: !ref <n_classes>
    activation_type: !ref <activation_type>
"""

myYAMLFilePath = "/content/benchmarks/benchmarks/MOABB/hparams/MotorImagery/BNCI2014001/MyVanillaNeuralNet.yaml"

# Write the class definition to the file
with open(myYAMLFilePath, 'w') as file:
    file.write(MyVanillaNeuralNetworkYAML)

# Model

In [ ]:
filePath="/content/benchmarks/benchmarks/MOABB/models/MyVanillaNeuralNet.py"
fileContent = '''
import torch
import speechbrain as sb

class MyVanillaNeuralNet(torch.nn.Module):
    """MyVanillaNeuralNet.

    A vanilla neural network (fully connected neural network) for EEG data classification.

    Arguments
    ---------
    input_shape : tuple
        The shape of the input tensor (batch, time, EEG channel, channel).
    hidden_layers : list of int
        A list specifying the number of neurons in each hidden layer.
    activation_type: str
        Activation function of the hidden layers.
    dropout : float
        Dropout probability.
    n_neurons : int
        Number of output neurons (classes).

    Example
    -------
    >>> inp_tensor = torch.rand([1, 200, 32, 1])
    >>> model = VanillaNeuralNet(input_shape=inp_tensor.shape, hidden_layers=[128, 64], activation="relu",dropout=0.5, n_neurons=4)
    >>> output = model(inp_tensor)
    >>> output.shape
    torch.Size([1, 4])
    """

    def __init__(
        self,
        input_shape=None,
        hidden_layers=[128, 64],
        dropout=0.5,
        n_neurons=4,
        activation_type="relu",
    ):
        super().__init__()

        if input_shape is None:
            raise ValueError("Must specify input_shape")

        # Flatten input shape to calculate input features for fully connected layers
        self.input_features = input_shape[1] * input_shape[2] * input_shape[3]

        # Create a list of layers
        layers = []
        if activation_type == "gelu":
            activation = torch.nn.GELU()
        elif activation_type == "relu":
            activation = torch.nn.ReLU()
        elif activation_type == "leaky_relu":
            activation = torch.nn.LeakyReLU()

        # Add hidden layers
        for i, hidden_units in enumerate(hidden_layers):
            if i == 0:
                # First layer
                layers.append(torch.nn.Linear(self.input_features, hidden_units))
            else:
                # Subsequent layers
                layers.append(torch.nn.Linear(hidden_layers[i - 1], hidden_units))

            # Add activation function
            layers.append(activation)

            # Add dropout
            layers.append(torch.nn.Dropout(dropout))

        # Add output layer
        layers.append(torch.nn.Linear(hidden_layers[-1], n_neurons))

        # Add softmax activation function
        layers.append(torch.nn.LogSoftmax(dim=1))

        # Define the layers as a sequential module
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        """Forward pass of the model.

        Arguments
        ---------
        x : torch.Tensor
            Input EEG tensor with shape (batch, time, EEG channel, channel).

        Returns
        -------
        predictions : torch.Tensor
            Model predictions.
        """
        # Flatten input tensor
        x = x.view(x.size(0), -1)

        # Pass the input through the fully connected layers
        x = self.layers(x)

        return x
'''
# Write the class definition to the file
with open(filePath, 'w') as file:
    file.write(fileContent)

# Hyperparameters Training

In [ ]:
%cd '/content/benchmarks/benchmarks/MOABB/'

!./run_hparam_optimization.sh --hparams 'hparams/MotorImagery/BNCI2014001/MyVanillaNeuralNet.yaml' \
--data_folder '/content/data/BNCI2014001'\
--cached_data_folder '/content/data' \
--output_folder '/content/results/hyperparameter-search/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--exp_name 'hyperparameter-search2' \
--nsbj_hpsearch 3 --nsess_hpsearch 2 \
--nruns_eval 1 \
--eval_metric acc \
--exp_max_trials 5 \
--number_of_epochs 100 \

# Training

In [ ]:
%cd '/content/benchmarks/benchmarks/MOABB/'

!./run_experiments.sh \
--hparams hparams/MotorImagery/BNCI2014001/MyVanillaNeuralNet.yaml \
--data_folder eeg_data \
--output_folder results/MotorImagery/BNCI2014001/MyVanillaNeuralNet \
--nsbj 9 \
--nsess 2 \
--nruns 1 \
--train_mode leave-one-session-out \
--device=cuda

Streaming output truncated to the last 5000 lines.
speechbrain.nnet.schedulers - Changing lr from 0.0033 to 0.0034
speechbrain.utils.train_logger - epoch: 176, lr: 3.26e-03 - train loss: 1.24e-01 - valid loss: 3.43, valid f1: 3.69e-01, valid acc: 3.75e-01, valid cm: [[6 5 2 1]
 [4 8 0 2]
 [6 3 4 1]
 [7 3 1 3]]
speechbrain.utils.epoch_loop - Going into epoch 177
speechbrain.nnet.schedulers - Changing lr from 0.0044 to 0.0044
speechbrain.utils.train_logger - epoch: 177, lr: 4.31e-03 - train loss: 4.53e-02 - valid loss: 3.56, valid f1: 3.51e-01, valid acc: 3.57e-01, valid cm: [[6 5 2 1]
 [4 7 0 3]
 [5 3 4 2]
 [6 3 2 3]]
speechbrain.utils.epoch_loop - Going into epoch 178
speechbrain.nnet.schedulers - Changing lr from 0.0046 to 0.0045
speechbrain.utils.train_logger - epoch: 178, lr: 4.65e-03 - train loss: 1.97e-01 - valid loss: 3.32, valid f1: 3.55e-01, valid acc: 3.75e-01, valid cm: [[7 5 1 1]
 [2 8 2 2]
 [4 4 4 2]
 [6 4 2 2]]
speechbrain.utils.epoch_loop - Going into epoch 179
speechbrai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
